In [260]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor

url = "https://api.hypixel.net/skyblock/auctions"

r = requests.get(url).json()
total_pages = int(r['totalPages'])
total_pages
df3 = pd.DataFrame()
def fetch_auction_house(page_number:int) -> pd.DataFrame:
        response = requests.get(f'{url}?page={page_number}')
        auctions_data = response.json()['auctions']
        df = pd.DataFrame(auctions_data)
        df.drop(columns=['auctioneer','profile_id','coop', 'extra','item_lore','tier','item_bytes','claimed_bidders','item_uuid','bids'],inplace=True)
        return df

def clean_dataframe(df:pd.DataFrame) -> pd.DataFrame:
        df['start'] = pd.to_datetime(df['start'], unit='ms')
        df['end'] = pd.to_datetime(df['end'], unit='ms')
        df['last_updated'] = pd.to_datetime(df['last_updated'], unit='ms')
        return df



for i in range(total_pages):
   df3 = pd.concat( [df3, fetch_auction_house(i)])

df = clean_dataframe(df3)

df

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
0,df4c2f41d16d472aaa6897b4a15152f8,2024-07-14 04:02:31.898,2024-07-14 10:02:31.898,Training Dummy,misc,294000,False,0,2024-07-14 04:02:31.898,True
1,dc95516691e04c5f84038da295c142a6,2024-07-14 04:02:31.666,2024-07-14 10:02:31.666,Experience Artifact,accessories,910000,False,0,2024-07-14 04:02:31.666,True
2,ac3d277a985b40d6b4050c9cb9e9de79,2024-07-14 04:02:31.588,2024-07-16 04:02:31.588,[Lvl 52] Mooshroom Cow,misc,8500000,False,0,2024-07-14 04:02:31.588,True
3,3d01f47184614567a7416203a2c3c5b5,2024-07-14 04:02:31.519,2024-07-16 04:02:31.519,Jaded Sorrow Boots,armor,13700000,False,0,2024-07-14 04:02:31.519,True
4,f83dca74698e45c7b8e49a17aefd5406,2024-07-14 04:02:31.300,2024-07-14 10:02:31.300,Velvet Top Hat,armor,84999,False,0,2024-07-14 04:02:31.300,True
...,...,...,...,...,...,...,...,...,...,...
574,16da74bf47f14517baa303c43d7bb757,2024-06-30 03:03:07.235,2024-07-14 05:03:07.235,Judgement Core,misc,406000000,False,0,2024-06-30 03:03:07.235,True
575,b554ea6158f1409fb742e6fd51c5eaa2,2024-06-30 02:38:49.438,2024-07-14 04:38:49.438,Plushie Reindeer Skin,misc,499999999,False,0,2024-06-30 02:38:49.438,True
576,4755bb995d68471d83f9de70d6a6f58d,2024-06-30 02:25:28.835,2024-07-14 04:25:28.835,Crochet Tiger Plushie,misc,26000000,False,0,2024-06-30 02:25:28.835,True
577,a7b5a37a604e47cc933d110a81c12c16,2024-06-30 02:25:06.798,2024-07-14 04:25:06.798,Barry Century Cake,misc,50000000,False,0,2024-06-30 02:25:06.798,True


In [303]:
# Using threading improved avg runtime from 25 - 30seconds to 3 - 4 (Thanks ChatGPT for the help)

url = "https://api.hypixel.net/skyblock/auctions"
r = requests.get(url).json()
total_pages = int(r['totalPages'])


# For loops through the total_number of pages on the market place, requesting JSON for each page
# Each page contains a max of 1000 Items
def fetch_auction_house(page_number: int) -> pd.DataFrame:
    response = requests.get(f'{url}?page={page_number}')
    auctions_data = response.json()['auctions']
    df = pd.DataFrame(auctions_data)
    return df
# Clean DataFrame
# Remove unnescessary code
# Change all timestamps from UNIX to yyyy/mm/dd hour/min/sec/ms format
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df.drop(columns=['auctioneer', 'profile_id', 'coop', 'extra', 'item_lore', 'tier', 'item_bytes', 'claimed_bidders', 'item_uuid', 'bids'], inplace=True)
    df['start'] = pd.to_datetime(df['start'], unit='ms')
    df['end'] = pd.to_datetime(df['end'], unit='ms')
    df['last_updated'] = pd.to_datetime(df['last_updated'], unit='ms')
    desired_order = ['item_name','uuid','starting_bid','highest_bid_amount','category', 'start','end','last_updated', 'claimed','bin']
    df = df[desired_order]
    return df


#ChatGPT helped me with this one, ill need someone to explain this to me
# Use ThreadPoolExecutor to fetch pages concurrently
def fetch_all_pages_concurrently(total_pages: int) -> pd.DataFrame:
    with ThreadPoolExecutor(max_workers=15) as executor:
        futures = [executor.submit(fetch_auction_house, i) for i in range(total_pages)]
        result = pd.concat([future.result() for future in futures], ignore_index=True)
    return result

df3 = fetch_all_pages_concurrently(total_pages)
df = clean_dataframe(df3)

df

,item_name,uuid,starting_bid,highest_bid_amount,category,start,end,last_updated,claimed,bin
0,Mithril Gauntlet,31f85f6a739e4b18aed0504967cf5778,500000,0,misc,2024-07-14 06:09:31.394,2024-07-14 12:09:31.394,2024-07-14 06:09:31.394,False,True
1,Lapis Armor Boots,1386be354c81430cbcb31774393410e5,10000,0,armor,2024-07-14 06:09:30.082,2024-07-14 12:09:30.082,2024-07-14 06:09:30.082,False,True
2,Rabbit Helmet,bd2665c9356c471093059cd6c164b689,975000,0,armor,2024-07-14 06:09:29.804,2024-07-16 06:09:29.804,2024-07-14 06:09:29.804,False,True
3,Griffin Upgrade Stone,90930c0687754f03a463fa60f4c8724a,3799000,0,misc,2024-07-14 06:09:29.645,2024-07-14 12:09:29.645,2024-07-14 06:09:29.645,False,True
4,Hollow Boots,68d498bd03e44339a56275aca66c4582,350000,0,armor,2024-07-14 06:09:29.348,2024-07-14 12:09:29.348,2024-07-14 06:09:29.348,False,True
...,...,...,...,...,...,...,...,...,...,...
58000,Ancient Diamond Necron Head ✪✪✪✪✪➍,846b2a0ef4a84e7aa4c8e4095e723c44,800000000,0,armor,2024-06-30 05:08:34.029,2024-07-14 07:08:34.029,2024-06-30 05:08:34.029,False,True
58001,Celestial Storm's Helmet Skin,5ee7280bb6384067bd06bba11e940875,700000000,0,misc,2024-06-30 05:07:50.362,2024-07-14 07:07:50.362,2024-06-30 05:07:50.362,False,True
58002,Celestial Storm's Helmet Skin,1870a1741716411da085cc20f8ab81a9,700000000,0,misc,2024-06-30 05:07:09.782,2024-07-14 07:07:09.782,2024-06-30 05:07:09.782,False,True
58003,✿ Ancient Necron's Chestplate ✪✪✪✪✪➋,6bf786429aac4cb198843957593b70a3,198790000,0,armor,2024-06-30 04:39:54.649,2024-07-14 06:39:54.649,2024-06-30 04:39:54.649,False,True


In [304]:
def find_item(item_name:str, specified_price:int) -> pd.DataFrame:
    filtered_df = df[df['item_name'] == item_name]
    filtered_df = filtered_df.sort_values(by=['starting_bid'])
    filtered_df = filtered_df.query(f'starting_bid < {specified_price} and bin == True and claimed == False')
    filtered_df.loc[:,'uuid'] = '/viewauction ' + filtered_df['uuid'].astype(str)
    if filtered_df.empty:
        return "Item not with specified Parameters"
    return filtered_df

find_item('Portal Dye', 79000000)

,item_name,uuid,starting_bid,highest_bid_amount,category,start,end,last_updated,claimed,bin
1019,Portal Dye,/viewauction 9dd065875b81479f9e736bffaa86808f,78900000,0,misc,2024-07-14 06:02:54.283,2024-07-14 12:02:54.283,2024-07-14 06:02:54.283,False,True
1204,Portal Dye,/viewauction 3a336be934b44d2c8db270fb58389b6d,78999999,0,misc,2024-07-14 06:01:43.060,2024-07-16 06:01:43.060,2024-07-14 06:01:43.060,False,True
895,Portal Dye,/viewauction 7bdd9c718ac0468586d1721602bc8c8d,78999999,0,misc,2024-07-14 06:03:37.378,2024-07-14 12:03:37.378,2024-07-14 06:03:37.378,False,True
544,Portal Dye,/viewauction 821748089f05443fa3d663d9dfe2a667,78999999,0,misc,2024-07-14 06:06:05.705,2024-07-16 06:06:05.705,2024-07-14 06:06:05.705,False,True
954,Portal Dye,/viewauction aa69d43999564109ac84bd9cd1cb032d,78999999,0,misc,2024-07-14 06:03:12.785,2024-07-16 06:03:12.785,2024-07-14 06:03:12.785,False,True


In [305]:
# Search Dictionary 
# USAGE
# Key = item_name
# Value = Maximum value of item
# search_dict = {
#   'Key': Value,
#   'Key': Value,
# }
search_dict = {
    'Portal Dye':78000000,
    'Molten Cloak': 900000,
    'Molten Belt': 900000		
    }

def find_item(item_name:str, specified_price:int) -> pd.DataFrame:
    filtered_df = df[df['item_name'] == item_name]
    filtered_df = filtered_df.sort_values(by=['starting_bid'])
    filtered_df = filtered_df.query(f'starting_bid < {specified_price} and bin == True and claimed == False')
    filtered_df.loc[:,'uuid'] = '/viewauction ' + filtered_df['uuid'].astype(str)
    if filtered_df.empty:
        return pd.DataFrame(columns=df.columns)
    return filtered_df

df4 = pd.DataFrame()

# Loop through Dictionary
for item_name,specified_price in search_dict.items():
    result = find_item(item_name,specified_price)
    df4 = pd.concat([df4,result], ignore_index=True)
df4

C:\Users\jdangelo\AppData\Local\Temp\ipykernel_24164\569055046.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df4 = pd.concat([df4,result], ignore_index=True)


,item_name,uuid,starting_bid,highest_bid_amount,category,start,end,last_updated,claimed,bin
0,Molten Cloak,/viewauction 99609ed3fc294b249f2b019484627178,899000,0,misc,2024-07-14 05:20:55.841,2024-07-14 11:20:55.841,2024-07-14 05:20:55.841,False,True
1,Molten Belt,/viewauction c182f9b5c8a747b698527d63b72921c3,800000,0,misc,2024-07-14 06:06:17.978,2024-07-14 12:06:17.978,2024-07-14 06:06:17.978,False,True
2,Molten Belt,/viewauction b3f41776b04e491090296cc2324b6159,800000,0,misc,2024-07-14 05:33:01.115,2024-07-16 05:33:01.115,2024-07-14 05:33:01.115,False,True
3,Molten Belt,/viewauction 0420ff52a4404213b4c8336c11b04867,800000,0,misc,2024-07-14 05:37:37.126,2024-07-14 11:37:37.126,2024-07-14 05:37:37.126,False,True
